# Tutorial with examples

In [1]:
from cmfa.fluxomics_data.reaction import Reaction, AtomPattern
from cmfa.fluxomics_data.reaction_network import ReactionNetwork
from cmfa.data_preparation import import_fluxomics_dataset_from_json
from cmfa.fluxomics_data.emu import EMU, EMUReaction


In [2]:
import os

CUR_DIR = os.getcwd()
TEST_DIR = f'{CUR_DIR}/../data/test_data'

TEST_DIR

'/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/../data/test_data'

In [3]:
model = import_fluxomics_dataset_from_json(f'{TEST_DIR}/model.json')

In [4]:
model.reaction_network.R4

Reaction id=R4, name=R4,stoichiometry={'B': {abc: -1.0}, 'C': {de: -1.0}, 'D': {bcd: 1.0}, 'E': {a: 1.0, e: 1.0}},reversible=False, 

In [5]:
print(model.reaction_network.reaction_adjacency_matrix())

compound           A         B     C         D               E           F
pattern          abc       abc    bc  de   abc   bcd         a     e   abc
compound pattern                                                          
A        abc      []      [R1]    []  []    []    []        []    []    []
B        abc      []        []  [R3]  []  [R2]  [R4]  [R3, R4]    []    []
C        bc       []        []    []  []    []    []        []    []    []
         de       []        []    []  []    []  [R4]        []  [R4]    []
D        abc      []  [R2_rev]    []  []    []    []        []    []  [R5]
         bcd      []        []    []  []    []    []        []    []    []
E        a        []        []    []  []    []    []        []    []    []
         e        []        []    []  []    []    []        []    []    []
F        abc      []        []    []  []    []    []        []    []    []


In [6]:
def atom_mapping_to_reaction(emu: EMU, reaction: Reaction, reverse: bool=False) -> list:
    """
    Given an EMU and reaction information, returns an EMUReaction object.
    """
    try:
        atom_transitions = reaction.stoichiometry_input[emu.compound].keys()
    except:
        raise ValueError(f"Cannot find the EMU compound in the corresponding reaction")

    emu_reactions = []
    # Use emu to find matching atoms in given compounds.
    # TODO: What if the emu matches two same compounds in reaction
    for transition in atom_transitions:
        adjusted_indices = [i - 1 for i in emu.atom_number]
        try:
            matched_atoms = ''.join(transition[i] for i in adjusted_indices)
        except:
            raise ValueError(f"Cannot find atoms in compound {emu.compound} with indices {emu.atom_number_input}")

        # Extracting the stoichiometry for the specific EMU
        emu_stoichiometry = {}
        emu_reactants = {emu}
        for cpd, stoich in reaction.stoichiometry_input.items():
            # new_stoich = {}
            for pattern, coeff in stoich.items():
                if cpd == emu.compound: # Add the original EMU
                    # new_stoich[emu.atom_number_input] = -coeff if reverse else coeff
                    # emu_stoichiometry[cpd] = new_stoich
                    emu_stoichiometry[emu.id] = -coeff if reverse else coeff

                elif (not reverse and coeff < 0) or (reverse and reaction.reversible and coeff > 0):
                    # For matched reactants or products in reverse reaction
                    reactant_matched_atom_number = _find_matchable_atoms(pattern, matched_atoms)
                    if len(reactant_matched_atom_number) != 0:
                        emu_id = f"{cpd}_{reactant_matched_atom_number}"
                        emu_reactants.add(EMU(id=emu_id, compound=cpd, atom_number_input=reactant_matched_atom_number))
                        # emu_stoichiometry.setdefault(cpd, {})[reactant_matched_atom_number] = -coeff if reverse else coeff
                        emu_stoichiometry[emu_id] = -coeff if reverse else coeff

        emu_reactions.append(EMUReaction(reaction_id=reaction.id, emu_stoichiometry=emu_stoichiometry))

    return emu_reactions, set(emu_reactants)

def _find_matchable_atoms(pattern: str, matched_atoms: str):
    
    """
    Finds matchable atoms between a pattern and matched_atoms, returning the positions as a string.
    """
    matched_pattern = [str(pattern.index(char) + 1) for char in matched_atoms if char in pattern]
    return ''.join(sorted(matched_pattern))


# Example Usage
product_emu = EMU(id="D_123", compound="D", atom_number_input="123")
reaction = Reaction(id='R4', name='R4', stoichiometry_input={'B': {"abc": -1.0}, 'C': {"de": -1.0}, 'D': {"bcd": 1.0}, 'E': {"a": 1.0, "e": 1.0}}, reversible=True)

emu_reaction = atom_mapping_to_reaction(product_emu, reaction, reverse=False)
print(emu_reaction)


([reaction id: R4, EMU stoichiometry: {'B_23': -1.0, 'C_1': -1.0, 'D_123': 1.0}], {EMU id: D_123, compound id: D, atom pattern: (1, 2, 3), EMU id: C_1, compound id: C, atom pattern: (1,), EMU id: B_23, compound id: B, atom pattern: (2, 3)})


In [7]:
def find_participated_reactions(cur_emu, MAM):
    """
    Identifies the reactions in which a given EMU participates within the Metabolite Adjacency Matrix (MAM).
    
    Parameters:
    - MAM: DataFrame representing the Metabolite Adjacency Matrix with multi-index columns.
    - cur_emu: The current EMU object, expected to have at least a 'compound' attribute.
    
    Returns:
    - A DataFrame filtered to show only the reactions (columns) in which the current EMU's compound participates.
      The cells are boolean, indicating whether the reaction is non-empty (True) or not (False).
    """
    # Extract reactions for the current EMU's compound
    participated_reactions = MAM.xs(cur_emu.compound, level=0, axis=1).map(
        lambda x: x if isinstance(x, list) and len(x) > 0 else [])
    
    all_participated_reactions = set(item for sublist in participated_reactions.values.flatten() for item in sublist)
    return all_participated_reactions


In [8]:
import pandas as pd
import re
from collections import deque

def decompose_network(
    target_EMU: EMU, reaction_network: ReactionNetwork
) -> pd.DataFrame:
    """
    Return a EMU map with a list of EMUs based on the target compound from the reaction network. 
    """
    MAM = reaction_network.reaction_adjacency_matrix()
    queue = deque([target_EMU])
    visited = set()
    emu_maps = {}
    emu_cpd_list = {}

    while queue:
        # Keep poping new EMU added from previous round.
        cur_emu = queue.popleft()
        # print(f"Current EMU: {cur_emu.compound}, have visited {visited}")
        if cur_emu in visited:
            continue
        visited.add(cur_emu)

        # Add new EMU map if the EMU size is not added
        emu_size = cur_emu.size
        if emu_size not in emu_maps:
            emu_maps[emu_size] = []
            emu_cpd_list[emu_size] = []
        
        # Find the reaction that is participated in the current emu
        participated_reactions = find_participated_reactions(cur_emu, MAM)
        new_map = []
        emus = []
        for r in participated_reactions:
            # Handle forward and reverse case
            # 
            if r.endswith('_rev'):
                new_emu_reactions, new_emus = atom_mapping_to_reaction(cur_emu, reaction_network.find_reaction_by_id(r[:-4]), reverse=True)
            else:
                new_emu_reactions, new_emus = atom_mapping_to_reaction(cur_emu, reaction_network.find_reaction_by_id(r), reverse=False)
            # print(new_emu_reactions, new_emus)
            new_map.append(new_emu_reactions)
            emus.append(new_emus)
            # TODO: Find equivalent EMUs
            for e in new_emus:
                if e not in visited and e not in queue:
                    queue.append(e)
                if e not in emu_cpd_list[emu_size]:
                    emu_cpd_list[emu_size] += [e]
        
        if len(new_map) > 0:
            # emu_maps[emu_size].setdefault(cur_emu, []).append(new_map)
            for i in new_map:
                emu_maps[emu_size] += i
            
    return emu_maps, emu_cpd_list



In [9]:
f_emu = EMU(id='F_123', compound='F', atom_number_input='123')
emu_network, emu_set = decompose_network(f_emu, model.reaction_network)
emu_set

{3: [EMU id: F_123, compound id: F, atom pattern: (1, 2, 3),
  EMU id: D_123, compound id: D, atom pattern: (1, 2, 3),
  EMU id: C_1, compound id: C, atom pattern: (1,),
  EMU id: B_23, compound id: B, atom pattern: (2, 3),
  EMU id: B_123, compound id: B, atom pattern: (1, 2, 3),
  EMU id: A_123, compound id: A, atom pattern: (1, 2, 3)],
 1: [EMU id: C_1, compound id: C, atom pattern: (1,),
  EMU id: B_2, compound id: B, atom pattern: (2,),
  EMU id: D_2, compound id: D, atom pattern: (2,),
  EMU id: A_2, compound id: A, atom pattern: (2,),
  EMU id: B_3, compound id: B, atom pattern: (3,),
  EMU id: D_3, compound id: D, atom pattern: (3,),
  EMU id: A_3, compound id: A, atom pattern: (3,)],
 2: [EMU id: D_23, compound id: D, atom pattern: (2, 3),
  EMU id: B_23, compound id: B, atom pattern: (2, 3),
  EMU id: A_23, compound id: A, atom pattern: (2, 3),
  EMU id: C_1, compound id: C, atom pattern: (1,),
  EMU id: B_3, compound id: B, atom pattern: (3,)]}

In [10]:
emu_network

{3: [reaction id: R5, EMU stoichiometry: {'D_123': -1.0, 'F_123': 1.0},
  reaction id: R4, EMU stoichiometry: {'B_23': -1.0, 'C_1': -1.0, 'D_123': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_123': -1.0, 'D_123': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_123': 1.0, 'D_123': -1.0},
  reaction id: R1, EMU stoichiometry: {'A_123': -1.0, 'B_123': 1.0}],
 1: [reaction id: R3, EMU stoichiometry: {'B_2': -1.0, 'C_1': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_2': 1.0, 'D_2': -1.0},
  reaction id: R1, EMU stoichiometry: {'A_2': -1.0, 'B_2': 1.0},
  reaction id: R4, EMU stoichiometry: {'B_3': -1.0, 'D_2': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_2': -1.0, 'D_2': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_3': 1.0, 'D_3': -1.0},
  reaction id: R1, EMU stoichiometry: {'A_3': -1.0, 'B_3': 1.0},
  reaction id: R4, EMU stoichiometry: {'C_1': -1.0, 'D_3': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_3': -1.0, 'D_3': 1.0}],
 2: [reaction id: R2, EMU stoichiometry: {'B_23': 

In [99]:
import pandas as pd
from sympy import Symbol, Mul

def create_emu_stoichiometry_matrix(emu_network, emu_size):

    # Create matrix indices
    reactants = set()
    products = set()

    for reaction in emu_network[emu_size]:
        # Directly extend the sets without checking the length
        reactants.update([" * ".join(reaction.reactants)])
        products.update([" * ".join(reaction.products)])

    # Combine reactants and products into a single tuple for output
    indices = tuple(reactants.union(products))
    
    # Initialize the DataFrame with indices for both rows and columns
    matrix = pd.DataFrame(index=indices, columns=indices, data=[])
    # Populate the matrix
    for reaction in emu_network[emu_size]: 
        row_key, col_key = None, None  # Initialize keys
        
        for emu_id, stoich in reaction.emu_stoichiometry.items():
            if stoich < 0:  # Reactant
                for idx in indices:
                    # Find the matching reactant row
                    if emu_id in idx.split(' * '):
                        row_key = idx
                        
            elif stoich > 0:  # Product
                for idx in indices:
                    # Find the matching product column
                    if emu_id in idx.split(' * '):
                        col_key = idx
                
            # Update the matrix cell with reaction ID and stoichiometry
            if row_key and col_key:
                cell_value = Mul(stoich, reaction.flux_symbol)
                matrix.at[row_key, col_key] = cell_value

    return matrix

matrix = create_emu_stoichiometry_matrix(emu_network, 1)
print(matrix)


     A_2        C_1         B_3  A_3         B_2        D_3        D_2
A_2  NaN        NaN         NaN  NaN   [1.0, R1]        NaN        NaN
C_1  NaN        NaN         NaN  NaN         NaN  [1.0, R4]        NaN
B_3  NaN        NaN         NaN  NaN         NaN  [1.0, R2]  [1.0, R4]
A_3  NaN        NaN   [1.0, R1]  NaN         NaN        NaN        NaN
B_2  NaN  [1.0, R3]         NaN  NaN         NaN        NaN  [1.0, R2]
D_3  NaN        NaN  [-1.0, R2]  NaN         NaN        NaN        NaN
D_2  NaN        NaN         NaN  NaN  [-1.0, R2]        NaN        NaN


In [103]:
import numpy as np

# Assuming you have a way to get the flux for a reaction ID
# For demonstration, here's a mock function that returns a flux value
def get_flux(reaction_id):
    # Mock flux values
    flux_values = {'R1': 0.5, 'R2': 0.8, 'R3': 0.6, 'R4': 0.7}
    return flux_values.get(reaction_id, 0)

# Function to process and multiply the stoichiometry by the reaction flux
def process_matrix(matrix):
    # Initialize a new matrix with zeros
    processed_matrix = np.zeros(matrix.shape)

    # Iterate over the matrix
    for i, row in matrix.iterrows():
        for j, cell in enumerate(row):
            if pd.isnull(cell) or cell == []:
                continue  # Skip NaN or empty cells
            
            # Initialize a variable to hold the product of stoichiometry and flux
            product = 1
            
            # If there are multiple items, multiply their fluxes with the stoichiometry
            for item in cell:
                stoich, reaction_id = item
                product *= stoich * get_flux(reaction_id)
            
            # Update the processed matrix with the computed product
            processed_matrix[i][j] = product

    return processed_matrix

# Convert the DataFrames to numpy arrays and process them
matrix_A_np = process_matrix(matrix_A)
matrix_B_np = process_matrix(matrix_B)

print(matrix_A_np)
print(matrix_B_np)



TypeError: 'float' object is not iterable

Matrix A after calculation:
     C_1  B_3  B_2   D_3   D_2
C_1 -3.0  NaN    0   NaN   NaN
B_3  NaN -5.0  NaN     0   NaN
B_2  NaN  NaN -5.0   NaN     0
D_3    0    0  NaN -12.0   NaN
D_2  NaN    0    0   NaN -12.0

Matrix B after calculation:
     A_2  A_3
C_1  NaN  NaN
B_3  NaN    0
B_2    0  NaN
D_3  NaN  NaN
D_2  NaN  NaN
